# Setup

This notebook illustrates how to access the Star Discrepancy problems using IOHexperimenter.
To install IOHexperimenter, the following command can be used:

In [1]:
#%pip install ioh

Please make sure that you use version >= 0.3.9 to follow the examples in this notebook.
Once installed, simply import the package

In [2]:
import ioh

# Accessing problems

In ioh, everything revolves around the problem class, so we start by creating an individual problem. For the star discrepancy, the problems can be represented as either continuous or discrete. We start with the continuous version, but the underlying problems are identical, and the interfaces are equivalent. The 2024 version of the competition uses the integer-version of these problems, which are described in the next section. 

To see which problems are available, we can check:

In [3]:
ioh.ProblemClass.STAR_REAL.problems

{30: 'UniformStarDiscrepancy10',
 31: 'UniformStarDiscrepancy25',
 32: 'UniformStarDiscrepancy50',
 33: 'UniformStarDiscrepancy100',
 34: 'UniformStarDiscrepancy150',
 35: 'UniformStarDiscrepancy200',
 36: 'UniformStarDiscrepancy250',
 37: 'UniformStarDiscrepancy500',
 38: 'UniformStarDiscrepancy750',
 39: 'UniformStarDiscrepancy1000',
 40: 'SobolStarDiscrepancy10',
 41: 'SobolStarDiscrepancy25',
 42: 'SobolStarDiscrepancy50',
 43: 'SobolStarDiscrepancy100',
 44: 'SobolStarDiscrepancy150',
 45: 'SobolStarDiscrepancy200',
 46: 'SobolStarDiscrepancy250',
 47: 'SobolStarDiscrepancy500',
 48: 'SobolStarDiscrepancy750',
 49: 'SobolStarDiscrepancy1000',
 50: 'HaltonStarDiscrepancy10',
 51: 'HaltonStarDiscrepancy25',
 52: 'HaltonStarDiscrepancy50',
 53: 'HaltonStarDiscrepancy100',
 54: 'HaltonStarDiscrepancy150',
 55: 'HaltonStarDiscrepancy200',
 56: 'HaltonStarDiscrepancy250',
 57: 'HaltonStarDiscrepancy500',
 58: 'HaltonStarDiscrepancy750',
 59: 'HaltonStarDiscrepancy1000'}

We can then construct a problem as follows:

In [4]:
f = ioh.get_problem(30, instance=1, dimension=5, problem_class=ioh.ProblemClass.STAR_REAL)

This problem contains all kinds of information about the way it is constructed. It also keeps track of all evaluations, which can be seen as follows:

In [5]:
print(f.meta_data)
print(f.state)

<MetaData: UniformStarDiscrepancy10 id: 30 iid: 1 dim: 5>
<State evaluations: 0 optimum_found: false current_best: <Solution x: [2.247116418577895e+308, 2.247116418577895e+308, 2.247116418577895e+308, 2.247116418577895e+308, 2.247116418577895e+308] y: -inf>>


Since this problem can be evaluated as a common python function, it should work directly with any optimizer. We make a basic random search to illustrate this:

In [6]:
import numpy as np

class RandomSearch:
    'Simple random search algorithm'
    def __init__(self, n: int, length: float = 0.0):
        self.n: int = n
        self.length: float = length
        
    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.uniform(problem.bounds.lb, problem.bounds.ub)
            self.length = np.linalg.norm(x)
            
            problem(x)   

If we want to run this algorithm on our problem, we can do the following:

In [7]:
r = RandomSearch(10)
r(f)
print(f.state)

<State evaluations: 10 optimum_found: false current_best: <Solution x: [0.8494103953454833, 0.31392058160556224, 0.6274687987418135, 0.6151237512038531, 0.6932407016816623] y: 0.07134705422944276>>


To run multiple independent runs on the same problem, we can reset the state as follows:


In [8]:
f.reset()

# Logging

The default usage of IOHExperimenter is in generating logs of benchmarking experiments which can be analyzed in IOHAnalyzer.

In [9]:
import os

logger = ioh.logger.Analyzer(
    root=os.getcwd(),                  # Store data in the current working directory
    folder_name="my-experiment",       # in a folder named: 'my-experiment'
    algorithm_name="random-search",    # meta-data for the algorithm used to generate these results
    store_positions=False               # disable storing x-variables in the logged files
)

# this automatically creates a folder 'my-experiment' in the current working directory
# if the folder already exists, it will given an additional number to make the name unique
logger

<Analyzer /home/jacob/code/IOHexperimenter/example/my-experiment>

We can add this logger to a problem so we can store the data when running our algorithm

In [10]:
f.attach_logger(logger)

In [11]:
r = RandomSearch(100)
r(f)

Once finished with the run, we can close the logger to force it to write the data (happens automatically when running as a python script)

In [12]:
logger.close()

# Experiment class

In Python, we provide the Experiment class which can be used to easily run a given algorithm over a larger number of problems.

In [13]:
experiment = ioh.Experiment(
    algorithm = RandomSearch(10), # An algorithm instance
    fids = list(ioh.ProblemClass.STAR_REAL.problems.keys())[:3],               # the id's of the problems we want to test
    dims = [2,5],                 # the dimensions of the problems we want to test
    iids = [1,2,3],               # the instance id's of the problems we want to test
    reps = 1,                     # the number of runs,
    problem_class=ioh.ProblemClass.STAR_REAL, #the problem type
    zip_output = True       
)

Running this experiment creates a zip-file, which can directly be processed by IOHanalyzer (https://iohanalyzer.liacs.nl)

In [14]:
experiment()

## Discrete version of the problems

In addition to the continuous problems, the star-discrepancy problem can also be considered as an integer optimization problem. Since we know that the optimial solution shares each of its coordinates with an exisiting grid-point, we can simply specify for each coordinate with the (sorted) grid points to use for the new solution. 

We can list the available problems in the same way as the continuous case

In [15]:
ioh.ProblemClass.STAR_INTEGER.problems

{30: 'UniformStarDiscrepancy10',
 31: 'UniformStarDiscrepancy25',
 32: 'UniformStarDiscrepancy50',
 33: 'UniformStarDiscrepancy100',
 34: 'UniformStarDiscrepancy150',
 35: 'UniformStarDiscrepancy200',
 36: 'UniformStarDiscrepancy250',
 37: 'UniformStarDiscrepancy500',
 38: 'UniformStarDiscrepancy750',
 39: 'UniformStarDiscrepancy1000',
 40: 'SobolStarDiscrepancy10',
 41: 'SobolStarDiscrepancy25',
 42: 'SobolStarDiscrepancy50',
 43: 'SobolStarDiscrepancy100',
 44: 'SobolStarDiscrepancy150',
 45: 'SobolStarDiscrepancy200',
 46: 'SobolStarDiscrepancy250',
 47: 'SobolStarDiscrepancy500',
 48: 'SobolStarDiscrepancy750',
 49: 'SobolStarDiscrepancy1000',
 50: 'HaltonStarDiscrepancy10',
 51: 'HaltonStarDiscrepancy25',
 52: 'HaltonStarDiscrepancy50',
 53: 'HaltonStarDiscrepancy100',
 54: 'HaltonStarDiscrepancy150',
 55: 'HaltonStarDiscrepancy200',
 56: 'HaltonStarDiscrepancy250',
 57: 'HaltonStarDiscrepancy500',
 58: 'HaltonStarDiscrepancy750',
 59: 'HaltonStarDiscrepancy1000'}

In [16]:
f = ioh.get_problem(30, instance=1, dimension=5, problem_class=ioh.ProblemClass.STAR_INTEGER)

The range of available integers scales with the number of grid-points, while the dimension remains the same as the continuous version. We can check this as follows (note that the bounds are inclusive):

In [17]:
f.bounds

<BoxConstraint lb: [[0, 0, 0, 0, 0]] ub: [[10, 10, 10, 10, 10]]>

In [18]:
f([10,0,10,10,10])

0.14038693698597768

In [19]:
class RandomSearch_int:
    'Simple random search algorithm'
    def __init__(self, n: int, length: float = 0.0):
        self.n: int = n
        self.length: float = length
        
    def __call__(self, problem: ioh.problem.IntegerSingleObjective) -> None:
        'Evaluate the problem n times with a randomly generated solution'
        
        for _ in range(self.n):
            # We can use the problems bounds accessor to get information about the problem bounds
            x = np.random.randint(problem.bounds.lb, problem.bounds.ub+1) #+1 since our upper bound is inclusive
            self.length = np.linalg.norm(x)
            
            problem(x)   

In [20]:
r = RandomSearch_int(100)
r(f)

In [21]:
f.state

<State evaluations: 101 optimum_found: false current_best: <Solution x: [8, 3, 10, 5, 9] y: 0.27935176246286275>>

## More information

For more background and other information on using IOHexperimenter, we refer to https://iohprofiler.github.io/IOHexp/

If you have any questions about the usage of the STAR_REAL discrepancy problems, or about IOHprofiler in general, please make an issue on https://github.com/IOHprofiler/IOHexperimenter

## Cleanup for CI

In [22]:
# cleanup
from shutil import rmtree
import os
import glob

def clean():
    for name in ("my-experiment", "ioh_data"):
        for path in glob.glob(f"{name}*"):
            if os.path.isfile(path):
                os.remove(path)
            if os.path.isdir(path):
                rmtree(path, ignore_errors=True)

def ls(p="./"):
    for obj in os.listdir(os.path.normpath(p)):
        print(obj)

def cat(f):
    with open(os.path.normpath(f)) as h:
        print(h.read())

clean()

rmtree("my-experiment", ignore_errors=True)
rmtree("ioh_data", ignore_errors=True)